# NLP project, creation d'un ChatBot

In [7]:
# we import all the libraries necessary for the project: Keras and NLTK
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [11]:
# we have now to tokenize work
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Lorena
[nltk_data]     G\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [14]:
# Now we will lemmatize each word and remove duplicate words from the list. 
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

48 documents
6 classes ['Thanks', 'goodbye', 'greeting', 'options', 'presentation', 'skills']
48 unique lemmatized words ["'s", ',', 'anyone', 'are', 'awesome', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'for', 'gongang', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'introduce', 'is', 'language', 'later', 'lorena', 'master', 'me', 'next', 'nice', 'programming', 'provide', 'see', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'use', 'what', 'which', 'who', 'you', 'yourself']


In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Lorena
[nltk_data]     G\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [16]:
#Now, we will create the training data in which we will provide the input and the output.
#Our input will be the pattern and output will be the class our input pattern belongs to. 
#But the computer doesn’t understand text so we will convert text into numbers.

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


Build a model

In [17]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
48/48 [==============================] - 1s 13ms/step - loss: 1.0437 - accuracy: 0.6667
Epoch 2/200
48/48 [==============================] - 0s 249us/step - loss: 0.0680 - accuracy: 1.0000
Epoch 3/200
48/48 [==============================] - 0s 251us/step - loss: 0.0106 - accuracy: 1.0000
Epoch 4/200
48/48 [==============================] - 0s 270us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 5/200
48/48 [==============================] - 0s 292us/step - loss: 0.0048 - accuracy: 1.0000
Epoch 6/200
48/48 [==============================] - 0s 312us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 7/200
48/48 [==============================] - 0s 313us/step - loss: 0.0019 - accuracy: 1.0000
Epoch 8/200
48/48 [==============================] - 0s 291us/step - loss: 6.5420e-04 - accuracy: 1.0000
Epoch 9/200
48/48 [==============================] - 0s 270us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 10/200
48/48 [==============================] - 0s 291us/step - loss: 8.3352e-04 -